In [1]:
import numpy as np
import pandas as pd

In [12]:
import geopandas as gpd

In [50]:
import os
import math
import logging
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely import wkt
from shapely.geometry import Point

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM  # Assuming these might be used later from original imports
from peft import PeftModel


In [2]:
df = pd.read_csv("DRIVES_data.csv")

/tmp/244673.tmpdir/ipykernel_572/2714555775.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("DRIVES_data.csv")


In [8]:
df_new = df.iloc[:-1] 

In [10]:
df_new.isna().sum(axis=0)

Unnamed: 0            0
participantId         0
vehicleName           0
tripStartDate         0
tripStartLatitude     0
tripStartLongitude    0
tripEndDate           0
tripEndLatitude       0
tripEndLongitude      0
dp                    0
gender                0
educ                  0
age                   0
dtype: int64

In [69]:
len(df_new.participantId.unique().tolist())

156

In [24]:
poi_df = pd.read_csv("../US_POI.csv")

In [25]:
poi_df

,PLACEKEY,PARENT_PLACEKEY,LOCATION_NAME,SAFEGRAPH_BRAND_IDS,BRANDS,STORE_ID,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,LATITUDE,...,ISO_COUNTRY_CODE,PHONE_NUMBER,OPEN_HOURS,CATEGORY_TAGS,OPENED_ON,CLOSED_ON,TRACKING_CLOSED_SINCE,GEOMETRY_TYPE,DOMAINS,WEBSITE
0,zzy-22r@8g7-475-brk,NaN,Troy Thomas ATL,NaN,NaN,NaN,Personal Care Services,"Hair, Nail, and Skin Care Services",81211,33.805047,...,US,1.404969e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,"[""troythomasatl.com""]",NaN
1,zzy-22s@5py-th9-z4v,NaN,Lawrence County Country Club,NaN,NaN,NaN,Other Amusement and Recreation Industries,Golf Courses and Country Clubs,713910,38.729712,...,US,1.618943e+10,NaN,Country Club,NaN,NaN,2019-07,POLYGON,[],NaN
2,zzy-22s@5vg-sqw-f2k,NaN,Dr. Kelsey Tumiel Psy.d,NaN,NaN,NaN,Offices of Physicians,"Offices of Physicians, Mental Health Specialists",621112,38.368681,...,US,1.209274e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
3,zzy-22s@62j-shz-3kf,22h-224@62j-shz-4qf,Greater Boston Tax & Accounting Service,NaN,NaN,NaN,Other Financial Investment Activities,Investment Advice,523930,42.354899,...,US,1.617787e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
4,zzy-22s@8dj-jv6-ks5,NaN,Jeffrey Berg,NaN,NaN,NaN,Activities Related to Real Estate,Residential Property Managers,531311,27.949076,...,US,1.727534e+10,"{ ""Mon"": [[""9:00"", ""17:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,2019-07,POLYGON,"[""jeffberghomes.com""]",http://jeffberghomes.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20309693,zzy-223@3bc-7cd-3h5,NaN,L & L's Bed & Breakfast,NaN,NaN,NaN,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110,61.137267,...,US,1.907835e+10,NaN,Breakfast,NaN,1900-01,2019-07,POLYGON,[],NaN
20309694,zzy-223@3bk-j6h-3kf,NaN,Kodiak Island Expeditions,NaN,NaN,NaN,Travel Arrangement and Reservation Services,Tour Operators,561520,57.824206,...,US,1.907513e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,"[""kodiakislandexpeditions.com""]",NaN
20309695,zzy-223@3bt-by5-gx5,NaN,Anchorage People Mover BENSON & SPENARD ESE,NaN,NaN,NaN,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,485113,61.193965,...,US,NaN,NaN,"Bus Station,Buses",NaN,NaN,2019-07,POINT,[],NaN
20309696,zzy-223@3bt-bym-d9z,zzz-222@3bt-byn-4n5,Abbott O'Rabbit Little League,NaN,NaN,NaN,Spectator Sports,Sports Teams and Clubs,711211,61.123440,...,US,1.907869e+10,"{ ""Mon"": [[""9:00"", ""17:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,2019-07,POLYGON,"[""abbottorabbit.org""]",NaN


In [70]:
df_new

,Unnamed: 0,participantId,vehicleName,tripStartDate,tripStartLatitude,tripStartLongitude,tripEndDate,tripEndLatitude,tripEndLongitude,dp,gender,educ,age
0,0,100119,1062501461,2022-07-01 00:00:32+00:00,38.556225,-90.262528,2022-07-01 00:04:20+00:00,38.556996,-90.275801,True,2.0,12.0,71.0
1,1,100133,1113004460,2022-07-01 00:04:29+00:00,38.568408,-90.449521,2022-07-01 00:20:37+00:00,38.567866,-90.477968,True,1.0,20.0,78.0
2,2,66598,1062504785,2022-07-01 00:05:18+00:00,28.406311,-81.469234,2022-07-01 00:09:38+00:00,28.406393,-81.473346,False,1.0,12.0,70.0
3,3,100126,1112902848,2022-07-01 00:05:20+00:00,38.719089,-90.372721,2022-07-01 00:08:15+00:00,38.719979,-90.368747,True,2.0,12.0,68.0
4,4,100142,1120105328,2022-07-01 00:11:59+00:00,38.508251,-90.338493,2022-07-01 00:36:54+00:00,38.437130,-90.376856,True,2.0,16.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240376,240376,100172,1112302116,2024-01-07 16:12:18+00:00,38.756815,-90.501772,2024-01-07 16:36:49+00:00,38.659597,-90.358018,True,2.0,18.0,68.0
240377,240377,68136,1062504949,2024-01-07 16:15:38+00:00,38.653459,-90.664819,2024-01-07 16:27:44+00:00,38.668698,-90.611867,False,1.0,18.0,73.0
240378,240378,100014,20585074,2024-01-07 16:15:41+00:00,38.614275,-90.540538,2024-01-07 16:19:48+00:00,38.600463,-90.531578,False,1.0,20.0,75.0
240379,240379,65272,1121601961,2024-01-07 16:16:02+00:00,38.548250,-91.243356,2024-01-07 16:31:57+00:00,38.554522,-91.352848,False,1.0,14.0,74.0


In [26]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from pyproj import Geod

# ---------- 1) Make a GeoDataFrame with start & end points (EPSG:4326) ----------
def make_points_gdf(df):
    df = df.copy()

    # Parse datetimes (assume UTC strings like "2022-07-01 00:00:32+00:00")
    for c in ["tripStartDate", "tripEndDate"]:
        df[c] = pd.to_datetime(df[c], utc=True, errors="coerce")

    # Build point geometries
    df["start_geom"] = [
        Point(lon, lat) if pd.notnull(lon) and pd.notnull(lat) else None
        for lon, lat in zip(df["tripStartLongitude"], df["tripStartLatitude"])
    ]
    df["end_geom"] = [
        Point(lon, lat) if pd.notnull(lon) and pd.notnull(lat) else None
        for lon, lat in zip(df["tripEndLongitude"], df["tripEndLatitude"])
    ]

    # Keep one active geometry (start); end is stored as an extra column
    gdf = gpd.GeoDataFrame(df, geometry="start_geom", crs="EPSG:4326")

    # Clean obvious nulls
    gdf = gdf.dropna(subset=["start_geom", "end_geom", "tripStartDate", "tripEndDate"])

    return gdf




In [31]:
gdf_pts = make_points_gdf(df_new)

In [34]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from pyproj import Geod

_geod = Geod(ellps="WGS84")

def _geo_dist_m(p1: Point, p2: Point) -> float:
    _, _, dist_m = _geod.inv(p1.x, p1.y, p2.x, p2.y)
    return dist_m

def stitch_trips_to_lines_with_gaps(
    gdf_points: gpd.GeoDataFrame,
    person_col: str = "participantId",
    join_dist_m: float = 100.0,
    join_gap_max: pd.Timedelta = pd.Timedelta("4H"),
    hard_break_gap: pd.Timedelta = pd.Timedelta("8H"),
):
    """
    Same logic as before, but also records the within-segment 'links' (gaps).
    Returns (gdf_lines, gdf_links), both EPSG:4326.
    """
    cols_needed = [person_col, "tripStartDate", "tripEndDate", "start_geom", "end_geom"]
    missing = [c for c in cols_needed if c not in gdf_points.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    seg_rows = []
    link_rows = []

    def _process_person(dfp: pd.DataFrame):
        dfp = dfp.sort_values("tripStartDate").reset_index(drop=True)

        seg_id = 0
        seg_points = []
        seg_trips = []
        seg_start_time = None
        seg_end_time = None
        last_end_point = None
        last_end_time = None

        # per-segment link accumulators
        seg_gaps_s = []
        seg_dists_m = []
        link_idx = 0

        def flush_segment(tag: str = "final"):
            nonlocal seg_id, seg_points, seg_trips, seg_start_time, seg_end_time
            nonlocal seg_gaps_s, seg_dists_m, link_idx

            if len(seg_points) >= 2:
                # summary stats over the links inside this segment
                link_count = len(seg_gaps_s)
                if link_count > 0:
                    max_gap_h = max(seg_gaps_s)/3600.0
                    mean_gap_h = (sum(seg_gaps_s)/link_count)/3600.0
                    total_gap_h = sum(seg_gaps_s)/3600.0
                else:
                    max_gap_h = mean_gap_h = total_gap_h = 0.0

                seg_rows.append({
                    person_col: dfp.iloc[0][person_col],
                    "seg_id": seg_id,
                    "num_trips": len(seg_trips),
                    "link_count": link_count,
                    "start_time": seg_start_time,
                    "end_time": seg_end_time,
                    "max_gap_h": max_gap_h,
                    "mean_gap_h": mean_gap_h,
                    "total_gap_h": total_gap_h,
                    "segment_type": tag,
                    "geometry": LineString(seg_points),
                })

            # reset for next segment
            seg_id += 1
            seg_points = []
            seg_trips = []
            seg_gaps_s = []
            seg_dists_m = []
            link_idx = 0

        for i, row in dfp.iterrows():
            s_pt, e_pt = row["start_geom"], row["end_geom"]
            s_t, e_t = row["tripStartDate"], row["tripEndDate"]

            if last_end_point is None:
                # start first segment
                seg_points = [s_pt, e_pt]
                seg_trips = [i]
                seg_start_time, seg_end_time = s_t, e_t
                last_end_point, last_end_time = e_pt, e_t
                continue

            gap = s_t - last_end_time
            dist = _geo_dist_m(last_end_point, s_pt)

            if pd.notnull(gap) and gap <= join_gap_max and dist <= join_dist_m:
                # continue this segment
                # record the link (prev end -> this start)
                gap_s = gap.total_seconds()
                link_rows.append({
                    person_col: dfp.iloc[0][person_col],
                    "seg_id": seg_id,
                    "link_idx": link_idx,
                    "stop_time": last_end_time,
                    "next_start_time": s_t,
                    "gap_s": gap_s,
                    "gap_h": gap_s/3600.0,
                    "dist_m": dist,
                    "geometry": LineString([last_end_point, s_pt]),
                })
                link_idx += 1
                seg_gaps_s.append(gap_s)
                seg_dists_m.append(dist)

                # extend segment with the new trip's end
                seg_points.append(e_pt)
                seg_trips.append(i)
                seg_end_time = e_t
                last_end_point, last_end_time = e_pt, e_t
            else:
                # break; classify soft vs hard
                tag = "soft_break" if (gap <= hard_break_gap and dist <= join_dist_m) else "hard_break"
                flush_segment(tag=tag)

                # start new segment with this trip
                seg_points = [s_pt, e_pt]
                seg_trips = [i]
                seg_start_time, seg_end_time = s_t, e_t
                last_end_point, last_end_time = e_pt, e_t

        # flush trailing segment
        flush_segment(tag="final")

    for _, dfp in gdf_points.groupby(person_col, sort=False):
        _process_person(dfp)

    gdf_lines = gpd.GeoDataFrame(seg_rows, geometry="geometry", crs="EPSG:4326")
    gdf_links = gpd.GeoDataFrame(link_rows, geometry="geometry", crs="EPSG:4326")
    return gdf_lines, gdf_links


/tmp/244673.tmpdir/ipykernel_572/2674153070.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  join_gap_max: pd.Timedelta = pd.Timedelta("4H"),
/tmp/244673.tmpdir/ipykernel_572/2674153070.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  hard_break_gap: pd.Timedelta = pd.Timedelta("8H"),


In [35]:
gdf_lines, gdf_links = stitch_trips_to_lines_with_gaps(
    gdf_pts,
    person_col="participantId",
    join_dist_m=100.0,
    join_gap_max=pd.Timedelta("4H"),
    hard_break_gap=pd.Timedelta("8H"),
)


/tmp/244673.tmpdir/ipykernel_572/636658557.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  join_gap_max=pd.Timedelta("4H"),
/tmp/244673.tmpdir/ipykernel_572/636658557.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  hard_break_gap=pd.Timedelta("8H"),


In [38]:
gdf_links

,participantId,seg_id,link_idx,stop_time,next_start_time,gap_s,gap_h,dist_m,geometry
0,100119,1,0,2022-07-01 14:06:34+00:00,2022-07-01 14:52:25+00:00,2751.0,0.764167,0.000000,"LINESTRING (-90.3808 38.50528, -90.3808 38.50528)"
1,100119,1,1,2022-07-01 15:07:23+00:00,2022-07-01 16:55:05+00:00,6462.0,1.795000,0.000000,"LINESTRING (-90.29917 38.55745, -90.29917 38.5..."
2,100119,1,2,2022-07-01 17:23:45+00:00,2022-07-01 18:03:37+00:00,2392.0,0.664444,0.000000,"LINESTRING (-90.27582 38.55702, -90.27582 38.5..."
3,100119,1,3,2022-07-01 18:09:01+00:00,2022-07-01 18:20:18+00:00,677.0,0.188056,0.000000,"LINESTRING (-90.29328 38.56542, -90.29328 38.5..."
4,100119,1,4,2022-07-01 18:26:11+00:00,2022-07-01 19:10:58+00:00,2687.0,0.746389,16.180535,"LINESTRING (-90.29914 38.5575, -90.29898 38.55..."
...,...,...,...,...,...,...,...,...,...
171369,63054,126,0,2023-12-29 22:23:02+00:00,2023-12-29 22:28:46+00:00,344.0,0.095556,0.000000,"LINESTRING (-89.58559 36.8905, -89.58559 36.8905)"
171370,63054,128,0,2023-12-31 16:37:26+00:00,2023-12-31 17:35:12+00:00,3466.0,0.962778,0.000000,"LINESTRING (-89.58264 36.88113, -89.58264 36.8..."
171371,63054,131,0,2024-01-03 15:13:39+00:00,2024-01-03 15:59:23+00:00,2744.0,0.762222,0.000000,"LINESTRING (-89.55116 37.09491, -89.55116 37.0..."
171372,63054,132,0,2024-01-05 23:41:35+00:00,2024-01-06 00:43:00+00:00,3685.0,1.023611,0.000000,"LINESTRING (-89.63902 37.36666, -89.63902 37.3..."


In [62]:
gdf_pts

,Unnamed: 0,participantId,vehicleName,tripStartDate,tripStartLatitude,tripStartLongitude,tripEndDate,tripEndLatitude,tripEndLongitude,dp,gender,educ,age,start_geom,end_geom
0,0,100119,1062501461,2022-07-01 00:00:32+00:00,38.556225,-90.262528,2022-07-01 00:04:20+00:00,38.556996,-90.275801,True,2.0,12.0,71.0,POINT (-90.26253 38.55623),POINT (-90.2758005 38.5569961)
1,1,100133,1113004460,2022-07-01 00:04:29+00:00,38.568408,-90.449521,2022-07-01 00:20:37+00:00,38.567866,-90.477968,True,1.0,20.0,78.0,POINT (-90.44952 38.56841),POINT (-90.4779678 38.5678664)
2,2,66598,1062504785,2022-07-01 00:05:18+00:00,28.406311,-81.469234,2022-07-01 00:09:38+00:00,28.406393,-81.473346,False,1.0,12.0,70.0,POINT (-81.46923 28.40631),POINT (-81.4733457 28.4063934)
3,3,100126,1112902848,2022-07-01 00:05:20+00:00,38.719089,-90.372721,2022-07-01 00:08:15+00:00,38.719979,-90.368747,True,2.0,12.0,68.0,POINT (-90.37272 38.71909),POINT (-90.3687474 38.7199791)
4,4,100142,1120105328,2022-07-01 00:11:59+00:00,38.508251,-90.338493,2022-07-01 00:36:54+00:00,38.437130,-90.376856,True,2.0,16.0,77.0,POINT (-90.33849 38.50825),POINT (-90.3768556 38.4371296)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240376,240376,100172,1112302116,2024-01-07 16:12:18+00:00,38.756815,-90.501772,2024-01-07 16:36:49+00:00,38.659597,-90.358018,True,2.0,18.0,68.0,POINT (-90.50177 38.75682),POINT (-90.3580179 38.6595973)
240377,240377,68136,1062504949,2024-01-07 16:15:38+00:00,38.653459,-90.664819,2024-01-07 16:27:44+00:00,38.668698,-90.611867,False,1.0,18.0,73.0,POINT (-90.66482 38.65346),POINT (-90.6118668 38.6686978)
240378,240378,100014,20585074,2024-01-07 16:15:41+00:00,38.614275,-90.540538,2024-01-07 16:19:48+00:00,38.600463,-90.531578,False,1.0,20.0,75.0,POINT (-90.54054 38.61427),POINT (-90.5315782 38.6004634)
240379,240379,65272,1121601961,2024-01-07 16:16:02+00:00,38.548250,-91.243356,2024-01-07 16:31:57+00:00,38.554522,-91.352848,False,1.0,14.0,74.0,POINT (-91.24336 38.54825),POINT (-91.3528483 38.5545217)


In [39]:
poi_df

,PLACEKEY,PARENT_PLACEKEY,LOCATION_NAME,SAFEGRAPH_BRAND_IDS,BRANDS,STORE_ID,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,LATITUDE,...,ISO_COUNTRY_CODE,PHONE_NUMBER,OPEN_HOURS,CATEGORY_TAGS,OPENED_ON,CLOSED_ON,TRACKING_CLOSED_SINCE,GEOMETRY_TYPE,DOMAINS,WEBSITE
0,zzy-22r@8g7-475-brk,NaN,Troy Thomas ATL,NaN,NaN,NaN,Personal Care Services,"Hair, Nail, and Skin Care Services",81211,33.805047,...,US,1.404969e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,"[""troythomasatl.com""]",NaN
1,zzy-22s@5py-th9-z4v,NaN,Lawrence County Country Club,NaN,NaN,NaN,Other Amusement and Recreation Industries,Golf Courses and Country Clubs,713910,38.729712,...,US,1.618943e+10,NaN,Country Club,NaN,NaN,2019-07,POLYGON,[],NaN
2,zzy-22s@5vg-sqw-f2k,NaN,Dr. Kelsey Tumiel Psy.d,NaN,NaN,NaN,Offices of Physicians,"Offices of Physicians, Mental Health Specialists",621112,38.368681,...,US,1.209274e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
3,zzy-22s@62j-shz-3kf,22h-224@62j-shz-4qf,Greater Boston Tax & Accounting Service,NaN,NaN,NaN,Other Financial Investment Activities,Investment Advice,523930,42.354899,...,US,1.617787e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
4,zzy-22s@8dj-jv6-ks5,NaN,Jeffrey Berg,NaN,NaN,NaN,Activities Related to Real Estate,Residential Property Managers,531311,27.949076,...,US,1.727534e+10,"{ ""Mon"": [[""9:00"", ""17:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,2019-07,POLYGON,"[""jeffberghomes.com""]",http://jeffberghomes.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20309693,zzy-223@3bc-7cd-3h5,NaN,L & L's Bed & Breakfast,NaN,NaN,NaN,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110,61.137267,...,US,1.907835e+10,NaN,Breakfast,NaN,1900-01,2019-07,POLYGON,[],NaN
20309694,zzy-223@3bk-j6h-3kf,NaN,Kodiak Island Expeditions,NaN,NaN,NaN,Travel Arrangement and Reservation Services,Tour Operators,561520,57.824206,...,US,1.907513e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,"[""kodiakislandexpeditions.com""]",NaN
20309695,zzy-223@3bt-by5-gx5,NaN,Anchorage People Mover BENSON & SPENARD ESE,NaN,NaN,NaN,Urban Transit Systems,Bus and Other Motor Vehicle Transit Systems,485113,61.193965,...,US,NaN,NaN,"Bus Station,Buses",NaN,NaN,2019-07,POINT,[],NaN
20309696,zzy-223@3bt-bym-d9z,zzz-222@3bt-byn-4n5,Abbott O'Rabbit Little League,NaN,NaN,NaN,Spectator Sports,Sports Teams and Clubs,711211,61.123440,...,US,1.907869e+10,"{ ""Mon"": [[""9:00"", ""17:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,2019-07,POLYGON,"[""abbottorabbit.org""]",NaN


In [41]:
import numpy as np
import pandas as pd
from shapely.geometry import LineString

# ---------- helpers ----------
def _resolve_col(df, candidates):
    """Find a column (case-insensitive) from a list of candidate names."""
    cols = {c.lower(): c for c in df.columns}
    for name in candidates:
        if name.lower() in cols:
            return cols[name.lower()]
    raise KeyError(f"None of {candidates} found in columns: {list(df.columns)}")

def _haversine_m(lat1, lon1, lat2, lon2):
    """
    Vectorized haversine distance in meters.
    lat/lon in degrees. lat1/lon1 can be scalars; lat2/lon2 can be numpy arrays.
    """
    R = 6371000.0  # meters
    lat1 = np.deg2rad(lat1); lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2); lon2 = np.deg2rad(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2.0*np.arcsin(np.sqrt(a))
    return R * c

def nearest_pois_for_links(gdf_links, poi_df, k=10, n_links=15,
                           person_col="participantId"):
    # Resolve POI columns (case-insensitive)
    lat_col  = _resolve_col(poi_df, ["LATITUDE", "latitude", "lat"])
    lon_col  = _resolve_col(poi_df, ["LONGITUDE", "longitude", "lon", "lng"])
    name_col = _resolve_col(poi_df, ["LOCATION_NAME", "location_name", "name"])
    top_col  = _resolve_col(poi_df, ["TOP_CATEGORY", "top_category"])
    sub_col  = _resolve_col(poi_df, ["SUB_CATEGORY", "sub_category"])
    key_col  = _resolve_col(poi_df, ["PLACEKEY", "placekey"])

    # Filter POIs with valid coords
    pois = poi_df.dropna(subset=[lat_col, lon_col]).copy()
    poi_lat = pois[lat_col].to_numpy(dtype=float)
    poi_lon = pois[lon_col].to_numpy(dtype=float)

    out_rows = []

    # Work on the first n_links
    links = gdf_links.iloc[:n_links].reset_index(drop=True)

    for i, row in links.iterrows():
        # Pull meta
        pid     = row.get(person_col, None)
        seg_id  = row.get("seg_id", None)
        link_id = row.get("link_idx", i)

        # Get start/end points from the LineString geometry
        geom = row.geometry
        if not isinstance(geom, LineString) or geom.is_empty:
            continue
        (start_lon, start_lat) = geom.coords[0]
        (end_lon,   end_lat)   = geom.coords[-1]

        # For each endpoint (start, end), compute distances to all POIs and take top-k
        for endpoint, (qlat, qlon) in [("start", (start_lat, start_lon)),
                                       ("end",   (end_lat,   end_lon))]:
            dists = _haversine_m(qlat, qlon, poi_lat, poi_lon)

            # Select k smallest efficiently, then sort
            k_eff = min(k, dists.size)
            idx_k = np.argpartition(dists, k_eff-1)[:k_eff]
            idx_sorted = idx_k[np.argsort(dists[idx_k])]

            # Record rows
            for rank, j in enumerate(idx_sorted, start=1):
                out_rows.append({
                    person_col: pid,
                    "seg_id": seg_id,
                    "link_idx": link_id,
                    "endpoint": endpoint,     # "start" or "end"
                    "rank": rank,             # 1..k
                    "poi_placekey": pois.iloc[j][key_col],
                    "poi_name": pois.iloc[j][name_col],
                    "top_category": pois.iloc[j][top_col],
                    "sub_category": pois.iloc[j][sub_col],
                    "poi_lat": float(pois.iloc[j][lat_col]),
                    "poi_lon": float(pois.iloc[j][lon_col]),
                    "distance_m": float(dists[j]),
                    # query point for reference
                    "q_lat": float(qlat),
                    "q_lon": float(qlon),
                })

    result = pd.DataFrame(out_rows).sort_values(
        ["participantId", "seg_id", "link_idx", "endpoint", "rank"],
        kind="stable"
    )
    return result

# ---------- run it ----------
# Make sure gdf_links is EPSG:4326 and poi_df has valid coords
nearest_15x10 = nearest_pois_for_links(gdf_links, poi_df, k=10, n_links=15,
                                       person_col="participantId")

# Peek
nearest_15x10.head(50)


,participantId,seg_id,link_idx,endpoint,rank,poi_placekey,poi_name,top_category,sub_category,poi_lat,poi_lon,distance_m,q_lat,q_lon
10,100119,1,0,end,1,227-229@5pj-24s-7bk,St Anthony's Cancer Care Center,Specialty (except Psychiatric and Substance Ab...,Specialty (except Psychiatric and Substance Ab...,38.505621,-90.381236,53.790099,38.505276,-90.380803
11,100119,1,0,end,2,22q-22c@5pj-24s-7bk,Jaclyn Reinagel NP,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505569,-90.380290,55.304197,38.505276,-90.380803
12,100119,1,0,end,3,22k-22d@5pj-24s-7bk,Kathleen Patel Fnp,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505562,-90.380280,55.559389,38.505276,-90.380803
13,100119,1,0,end,4,223-22f@5pj-24s-7bk,"MS. Kristin Wald Rnc, Whnp",Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505576,-90.380272,57.028847,38.505276,-90.380803
14,100119,1,0,end,5,228-224@5pj-24s-7bk,Roy Morris MD,Offices of Physicians,Offices of Physicians (except Mental Health Sp...,38.505571,-90.380266,57.131374,38.505276,-90.380803
15,100119,1,0,end,6,zzy-222@5pj-24s-4sq,Physicians Office Center,Offices of Physicians,Offices of Physicians (except Mental Health Sp...,38.505521,-90.380172,61.329125,38.505276,-90.380803
16,100119,1,0,end,7,223-22d@5pj-24s-7bk,Janice Eisleben Whnp,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505625,-90.380246,62.130804,38.505276,-90.380803
17,100119,1,0,end,8,222-23v@5pj-24s-7h5,Dr. Scott Zoeller DDS,Offices of Dentists,Offices of Dentists,38.505580,-90.380192,63.041205,38.505276,-90.380803
18,100119,1,0,end,9,229-222@5pj-24s-4sq,Midwest Oral Maxillofacial & Implant Surgery,Offices of Dentists,Offices of Dentists,38.505600,-90.380206,63.258133,38.505276,-90.380803
19,100119,1,0,end,10,223-224@5pj-24s-4sq,Mercy Clinic Gastroenterology 10012 Kennerly,General Medical and Surgical Hospitals,General Medical and Surgical Hospitals,38.505792,-90.380405,67.065397,38.505276,-90.380803


In [43]:
import geopandas as gpd
import pandas as pd

# --- 0) Assumptions ---
# gdf_pts has EPSG:4326 and two point columns: start_geom, end_geom
# poi_df has columns LATITUDE/LONGITUDE (or similar)

# --- 1) Build a single GeoSeries of all points (start + end) ---
all_pts = pd.concat([gdf_pts["start_geom"], gdf_pts["end_geom"]], ignore_index=True).dropna()
all_pts = gpd.GeoSeries(all_pts, crs="EPSG:4326")

# (optional but recommended) project to a planar CRS before computing hull,
# then convert back to WGS84 to avoid geodesic distortions.
# For US-wide data, EPSG:5070 (NAD83 / Conus Albers) works well.
all_pts_proj = all_pts.to_crs(5070)
hull_proj = all_pts_proj.unary_union.convex_hull
hull_4326 = gpd.GeoSeries([hull_proj], crs=5070).to_crs(4326).iloc[0]

# --- 2) Turn POIs into a GeoDataFrame ---
def _resolve_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in cols:
            return cols[c.lower()]
    raise KeyError(f"Need one of {candidates}")

lat_col = _resolve_col(poi_df, ["LATITUDE","latitude","lat"])
lon_col = _resolve_col(poi_df, ["LONGITUDE","longitude","lon","lng"])

poi_gdf = gpd.GeoDataFrame(
    poi_df.copy(),
    geometry=gpd.points_from_xy(poi_df[lon_col], poi_df[lat_col]),
    crs="EPSG:4326"
)

# --- 3) (Fast pre-filter) Clip by hull bounding box, then precise 'within' test ---
minx, miny, maxx, maxy = hull_4326.bounds
candidate_pois = poi_gdf.cx[minx:maxx, miny:maxy]   # uses spatial index when available

# precise point-in-polygon
pois_in_hull = candidate_pois[candidate_pois.within(hull_4326)].copy()

print(f"POIs total: {len(poi_gdf):,}")
print(f"POIs in hull: {len(pois_in_hull):,}")

# If you want the hull as a one-row GeoDataFrame for plotting / saving:
hull_gdf = gpd.GeoDataFrame({"name":["trip_convex_hull"]}, geometry=[hull_4326], crs="EPSG:4326")


/tmp/244673.tmpdir/ipykernel_572/2222590706.py:16: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  hull_proj = all_pts_proj.unary_union.convex_hull


POIs total: 20,309,698
POIs in hull: 12,143,934


In [46]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
# 4) Move to GPU/CPU and set eval mode:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 1) Where you saved your LoRA adapters after training:
CHECKPOINT = "/storage1/fs1/nlin/Active/sizhe/FO_DATA/checkpoint-dir/checkpoint-551"

# 2) Load the tokenizer from that folder (it contains tokenizer.json + vocab.txt)
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

base_name = "google/gemma-3-1b-it"     # <— exactly what you passed in your training script
base = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    attn_implementation="eager"
).to(device)

# 3) Now graft the adapter weights:
llm_model = PeftModel.from_pretrained(base, CHECKPOINT)


llm_model.to(device).eval()



PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForCausalLM(
      (model): Gemma3TextModel(
        (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma3DecoderLayer(
            (self_attn): Gemma3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1152, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1152, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
         

In [47]:
def embed_texts(texts: list[str]) -> torch.Tensor:
    """
    texts: a Python list of strings (each may contain literal "[sep]").
    example: ["[Museums, Historical Sites,...<sep>Museums, Historical Sites<sep>Natural History museum]",...]
    In the format of [<category><sep><subcategory><sep><name>]
    Returns: a GPU tensor of shape (len(texts), hidden_size) containing the
             final‐token embedding for each string.
    """
    # 2.1) Clean & force everything to str, replacing None/NaN with ""
    clean_texts=[]
    for t in texts:
        if t is None:
            clean_texts.append("")
        else:
            clean_texts.append(str(t))
    sep = tokenizer.sep_token  # e.g. "[SEP]" for BERT‐style; whatever your tokenizer.sep_token is
    clean_texts = [t.replace("[sep]", sep) for t in clean_texts]
    
    # 2.3) Batch‐tokenize all strings onto GPU
    enc = tokenizer(
        clean_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
        return_token_type_ids=False,
    ).to(device)
    #print("Batch input_ids are on:", enc.input_ids.device)
    # Now enc.input_ids and enc.attention_mask live on GPU.

    # 2.4) Forward‐pass (no gradients) to get hidden states
    with torch.no_grad():
        outputs = llm_model.base_model(
            input_ids=enc.input_ids,
            attention_mask=enc.attention_mask,
            output_hidden_states=True
        )
        # `outputs.hidden_states` is a tuple of length (num_layers+1);
        # each element has shape (batch_size, seq_len, hidden_size).
        last_hidden = outputs.hidden_states[-1]  # (batch_size, seq_len, hidden_size) on GPU

    # 2.5) For each sequence, pick out the final non‐pad token embedding
    seq_lens = enc.attention_mask.sum(dim=1) - 1  # (batch_size,) on GPU, index of last non-pad
    batch_size, hidden_size = last_hidden.size(0), last_hidden.size(2)

    # Gather the embedding at position (i, seq_lens[i], :)
    final_embs = last_hidden[torch.arange(batch_size), seq_lens, :]  # (batch_size, hidden_size) on GPU
    #print((final_embs))
    return final_embs


In [48]:
DEVICE     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
null_emb   = embed_texts(["<null_val>[sep]<null_val>"]) 
# null_emb is already a tensor
print(type(null_emb), null_emb.shape, null_emb.device)

# add the batch‐dim, cast & move to DEVICE
null_tensor = null_emb.float().unsqueeze(0).to(DEVICE)
print(null_tensor, null_tensor.device)

<class 'torch.Tensor'> torch.Size([1, 1152]) cuda:0
tensor([[[ 1.4538, -8.0599,  0.6345,  ..., -0.2421,  0.9947,  0.2485]]],
       device='cuda:0') cuda:0


In [51]:

ckpt = torch.load("bottleneck_mlp_newdata.pth",map_location = "cuda",weights_only=False)

raw_classes = ckpt.get("classes", None)
# Determine number of classes
n_old = len(raw_classes) if raw_classes is not None else ckpt["model_state_dict"]["head.bias"].shape[0]

# Convert classes to Python list
if raw_classes is None:
    classes = [f"class_{{i}}" for i in range(n_old)]
elif isinstance(raw_classes, np.ndarray):
    classes = raw_classes.tolist()
else:
    classes = list(raw_classes)
# Recreate and load your original BottleneckMLP
class BottleneckMLP(nn.Module):
    def __init__(self, in_dim, hid_dim, lat_dim, n_cls):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, lat_dim),
            nn.ReLU()
        )
        self.head = nn.Linear(lat_dim, n_cls)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.head(z)
        return z, logits

# Extract dims from checkpoint
in_dim     = ckpt["model_state_dict"]["encoder.0.weight"].shape[1]
hid_dim    = ckpt["model_state_dict"]["encoder.0.weight"].shape[0]
lat_dim    = ckpt["model_state_dict"]["encoder.2.weight"].shape[0]

# Instantiate and load weights
bottleneck_model = BottleneckMLP(in_dim, hid_dim, lat_dim, n_old).to(device)
bottleneck_model.load_state_dict(ckpt["model_state_dict"])
bottleneck_model.eval()



BottleneckMLP(
  (encoder): Sequential(
    (0): Linear(in_features=1152, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
  )
  (head): Linear(in_features=64, out_features=751, bias=True)
)

In [55]:
raw_classes

array(['<null_val>[sep]<null_val>',
       'Accounting, Tax Preparation, Bookkeeping, and Payroll Services[sep]<null_val>',
       'Accounting, Tax Preparation, Bookkeeping, and Payroll Services[sep]Accounting, Tax Preparation, Bookkeeping, and Payroll Services',
       'Accounting, Tax Preparation, Bookkeeping, and Payroll Services[sep]Offices of Certified Public Accountants',
       'Accounting, Tax Preparation, Bookkeeping, and Payroll Services[sep]Other Accounting Services',
       'Accounting, Tax Preparation, Bookkeeping, and Payroll Services[sep]Payroll Services',
       'Accounting, Tax Preparation, Bookkeeping, and Payroll Services[sep]Tax Preparation Services',
       'Activities Related to Credit Intermediation[sep]<null_val>',
       'Activities Related to Credit Intermediation[sep]Financial Transactions Processing, Reserve, and Clearinghouse Activities',
       'Activities Related to Credit Intermediation[sep]Mortgage and Nonmortgage Loan Brokers',
       'Activities Relat

In [53]:
bottleneck_model.eval()
with torch.no_grad():
    # Option A: get both z and logits
    z_null, logits_null = bottleneck_model(null_tensor)

    # Option B: just run the encoder if you don’t care about the head
    # z_null = model.encoder(null_tensor)

# 3) Pull back to numpy and squeeze off the batch‐dim
z_null = z_null.cpu().numpy().squeeze(0)  # shape: (LATENT_DIM,)

print("Null bottleneck code:", z_null)
null_label = "<null_val>[sep]<null_val>"


Null bottleneck code: [[  0.         12.700647   47.651257   66.52407    24.006102    1.296096
    0.          0.          7.9494076  85.64789    40.38713    69.084435
   38.75026   127.5442     29.369228   36.508415   63.713703   24.525366
    0.         58.351707   29.544384   66.118355   44.221233    0.
   65.13873   116.61834    82.30747    91.20707    65.97595    47.44527
   17.986979   44.931305   13.504261    0.         20.502945    8.2769
   26.23781    42.675663  151.97281     0.          0.         38.064827
   58.02574    23.736341   61.642185  112.67966     0.        154.26202
   31.802727    0.         70.91482    43.32689     0.         69.77493
   68.97989    41.14164    53.122627   71.48492    20.942825   24.69298
    9.73867    30.967611   46.18281    47.502865 ]]


In [54]:
idx = 0

# 2) grab the weight‐row (and optional bias)
with torch.no_grad():
    w_cls = bottleneck_model.head.weight[idx]     # torch.Tensor of shape (latent_dim,)
    b_cls = bottleneck_model.head.bias[idx]       # scalar


In [77]:
# All motions in each day - description with LLM?
# Intra-trajectory
# Inter-trajectory - summary of transition pattern
gdf_lines

,participantId,seg_id,num_trips,link_count,start_time,end_time,max_gap_h,mean_gap_h,total_gap_h,segment_type,geometry
0,100119,0,1,0,2022-07-01 00:00:32+00:00,2022-07-01 00:04:20+00:00,0.000000,0.000000,0.000000,hard_break,"LINESTRING (-90.26253 38.55623, -90.2758 38.557)"
1,100119,1,12,11,2022-07-01 13:45:26+00:00,2022-07-01 23:05:29+00:00,2.620000,0.674646,7.421111,hard_break,"LINESTRING (-90.2758 38.557, -90.3808 38.50528..."
2,100119,2,9,8,2022-07-02 15:28:32+00:00,2022-07-03 00:40:16+00:00,3.848333,0.989479,7.915833,hard_break,"LINESTRING (-90.27575 38.55701, -90.32059 38.5..."
3,100119,3,6,5,2022-07-03 13:57:05+00:00,2022-07-03 21:10:23+00:00,2.728889,1.358444,6.792222,hard_break,"LINESTRING (-90.27577 38.55699, -90.27839 38.5..."
4,100119,4,3,2,2022-07-04 01:26:54+00:00,2022-07-04 03:34:24+00:00,1.858333,0.955278,1.910556,hard_break,"LINESTRING (-90.27574 38.55698, -90.29867 38.5..."
...,...,...,...,...,...,...,...,...,...,...,...
69002,63054,129,1,0,2024-01-03 01:01:00+00:00,2024-01-03 01:09:12+00:00,0.000000,0.000000,0.000000,hard_break,"LINESTRING (-89.57807 36.86683, -89.55572 36.8..."
69003,63054,130,1,0,2024-01-03 02:01:45+00:00,2024-01-03 02:22:50+00:00,0.000000,0.000000,0.000000,hard_break,"LINESTRING (-89.56481 36.88344, -89.57809 36.8..."
69004,63054,131,2,1,2024-01-03 14:50:27+00:00,2024-01-03 16:25:08+00:00,0.762222,0.762222,0.762222,hard_break,"LINESTRING (-89.57809 36.86682, -89.55116 37.0..."
69005,63054,132,3,2,2024-01-05 22:55:50+00:00,2024-01-06 02:49:47+00:00,1.354444,1.189028,2.378056,hard_break,"LINESTRING (-89.57803 36.86687, -89.63902 37.3..."


In [79]:
# Involvement in each day, transition from POI to POI, what pattern in each stay

nearest_15x10.head(20)

,participantId,seg_id,link_idx,endpoint,rank,poi_placekey,poi_name,top_category,sub_category,poi_lat,poi_lon,distance_m,q_lat,q_lon,class_key,class_idx,class_vec_64
10,100119,1,0,end,1,227-229@5pj-24s-7bk,St Anthony's Cancer Care Center,Specialty (except Psychiatric and Substance Ab...,Specialty (except Psychiatric and Substance Ab...,38.505621,-90.381236,53.790099,38.505276,-90.380803,Specialty (except Psychiatric and Substance Ab...,664,"[-0.13361813, -0.33008018, -0.3315121, -0.0346..."
11,100119,1,0,end,2,22q-22c@5pj-24s-7bk,Jaclyn Reinagel NP,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505569,-90.380290,55.304197,38.505276,-90.380803,Offices of Other Health Practitioners[sep]Offi...,443,"[0.037678607, 0.07330139, 0.012516715, -0.1252..."
12,100119,1,0,end,3,22k-22d@5pj-24s-7bk,Kathleen Patel Fnp,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505562,-90.380280,55.559389,38.505276,-90.380803,Offices of Other Health Practitioners[sep]Offi...,443,"[0.037678607, 0.07330139, 0.012516715, -0.1252..."
13,100119,1,0,end,4,223-22f@5pj-24s-7bk,"MS. Kristin Wald Rnc, Whnp",Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505576,-90.380272,57.028847,38.505276,-90.380803,Offices of Other Health Practitioners[sep]Offi...,443,"[0.037678607, 0.07330139, 0.012516715, -0.1252..."
14,100119,1,0,end,5,228-224@5pj-24s-7bk,Roy Morris MD,Offices of Physicians,Offices of Physicians (except Mental Health Sp...,38.505571,-90.380266,57.131374,38.505276,-90.380803,Offices of Physicians[sep]Offices of Physician...,451,"[-0.025403716, -0.06948231, -0.008016454, 0.11..."
15,100119,1,0,end,6,zzy-222@5pj-24s-4sq,Physicians Office Center,Offices of Physicians,Offices of Physicians (except Mental Health Sp...,38.505521,-90.380172,61.329125,38.505276,-90.380803,Offices of Physicians[sep]Offices of Physician...,451,"[-0.025403716, -0.06948231, -0.008016454, 0.11..."
16,100119,1,0,end,7,223-22d@5pj-24s-7bk,Janice Eisleben Whnp,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505625,-90.380246,62.130804,38.505276,-90.380803,Offices of Other Health Practitioners[sep]Offi...,443,"[0.037678607, 0.07330139, 0.012516715, -0.1252..."
17,100119,1,0,end,8,222-23v@5pj-24s-7h5,Dr. Scott Zoeller DDS,Offices of Dentists,Offices of Dentists,38.505580,-90.380192,63.041205,38.505276,-90.380803,Offices of Dentists[sep]Offices of Dentists,441,"[-0.2805427, 0.003874096, 0.04817797, 0.108755..."
18,100119,1,0,end,9,229-222@5pj-24s-4sq,Midwest Oral Maxillofacial & Implant Surgery,Offices of Dentists,Offices of Dentists,38.505600,-90.380206,63.258133,38.505276,-90.380803,Offices of Dentists[sep]Offices of Dentists,441,"[-0.2805427, 0.003874096, 0.04817797, 0.108755..."
19,100119,1,0,end,10,223-224@5pj-24s-4sq,Mercy Clinic Gastroenterology 10012 Kennerly,General Medical and Surgical Hospitals,General Medical and Surgical Hospitals,38.505792,-90.380405,67.065397,38.505276,-90.380803,General Medical and Surgical Hospitals[sep]Gen...,261,"[-0.20431426, 0.14618827, -0.18811065, -0.2437..."


In [60]:
import pandas as pd
import numpy as np
import torch

SEP  = "[sep]"
NULL = "<null_val>"
def norm_token(x):
    # robust: works for NaN/None/numbers/strings
    if pd.isna(x):
        return NULL
    s = str(x).strip()
    return s if s else NULL

# build class_key safely (vectorized)
nearest_15x10 = nearest_15x10.copy()
nearest_15x10["class_key"] = (
    nearest_15x10["top_category"].apply(norm_token) + 
    SEP + 
    nearest_15x10["sub_category"].apply(norm_token)
)

# (optional but recommended) normalize raw_classes the same way to avoid mismatches
def normalize_key(k):
    if isinstance(k, str):
        parts = k.split(SEP)
        t = norm_token(parts[0] if len(parts)>0 else None)
        s = norm_token(parts[1] if len(parts)>1 else None)
        return f"{t}{SEP}{s}"
    return k

raw_classes_norm = [normalize_key(k) for k in raw_classes]

# lookup index for each key
cls_to_idx = {k:i for i,k in enumerate(raw_classes_norm)}
nearest_15x10["class_idx"] = nearest_15x10["class_key"].map(cls_to_idx).astype("Int64")

# attach 64-D class vectors from model.head.weight
W = bottleneck_model.head.weight.detach().cpu().numpy()                  # (C,64)
W = W / (np.linalg.norm(W, axis=1, keepdims=True) + 1e-12)    # normalize

nearest_15x10["class_vec_64"] = nearest_15x10["class_idx"].apply(
    lambda idx: None if pd.isna(idx) else W[int(idx)].astype(np.float32)
)

In [61]:
nearest_15x10

,participantId,seg_id,link_idx,endpoint,rank,poi_placekey,poi_name,top_category,sub_category,poi_lat,poi_lon,distance_m,q_lat,q_lon,class_key,class_idx,class_vec_64
10,100119,1,0,end,1,227-229@5pj-24s-7bk,St Anthony's Cancer Care Center,Specialty (except Psychiatric and Substance Ab...,Specialty (except Psychiatric and Substance Ab...,38.505621,-90.381236,53.790099,38.505276,-90.380803,Specialty (except Psychiatric and Substance Ab...,664,"[-0.13361813, -0.33008018, -0.3315121, -0.0346..."
11,100119,1,0,end,2,22q-22c@5pj-24s-7bk,Jaclyn Reinagel NP,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505569,-90.380290,55.304197,38.505276,-90.380803,Offices of Other Health Practitioners[sep]Offi...,443,"[0.037678607, 0.07330139, 0.012516715, -0.1252..."
12,100119,1,0,end,3,22k-22d@5pj-24s-7bk,Kathleen Patel Fnp,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505562,-90.380280,55.559389,38.505276,-90.380803,Offices of Other Health Practitioners[sep]Offi...,443,"[0.037678607, 0.07330139, 0.012516715, -0.1252..."
13,100119,1,0,end,4,223-22f@5pj-24s-7bk,"MS. Kristin Wald Rnc, Whnp",Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,38.505576,-90.380272,57.028847,38.505276,-90.380803,Offices of Other Health Practitioners[sep]Offi...,443,"[0.037678607, 0.07330139, 0.012516715, -0.1252..."
14,100119,1,0,end,5,228-224@5pj-24s-7bk,Roy Morris MD,Offices of Physicians,Offices of Physicians (except Mental Health Sp...,38.505571,-90.380266,57.131374,38.505276,-90.380803,Offices of Physicians[sep]Offices of Physician...,451,"[-0.025403716, -0.06948231, -0.008016454, 0.11..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,100119,2,3,start,6,zzw-225@5pk-n8z-73q,Schnucks,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,38.549330,-90.336102,61.366833,38.549646,-90.336681,Grocery Stores[sep]Supermarkets and Other Groc...,273,"[-0.05586769, -0.0036085523, -0.16025333, -0.1..."
286,100119,2,3,start,7,zzw-222@5pk-n8z-73q,Affton Plaza,Office Administrative Services,Office Administrative Services,38.549530,-90.335766,80.593320,38.549646,-90.336681,Office Administrative Services[sep]Office Admi...,436,"[0.063297324, -0.0022383793, -0.0053743916, -0..."
287,100119,2,3,start,8,225-222@5pk-n8z-73q,Summer Nails,Personal Care Services,Nail Salons,38.549384,-90.335655,93.836974,38.549646,-90.336681,Personal Care Services[sep]Nail Salons,557,"[0.067884274, 0.012117231, -0.3210142, 0.02714..."
288,100119,2,3,start,9,224-222@5pk-n8z-73q,Great Clips,Personal Care Services,Beauty Salons,38.549383,-90.335624,96.437238,38.549646,-90.336681,Personal Care Services[sep]Beauty Salons,554,"[-0.08265969, -0.16976076, -0.28624243, -0.025..."
